# Лабораторная работа по Теории кодирования
## Лабораторная работа №2
Выполнили студенты группы 6401-010302D

Баловнева Юлия

Богатырев Дмитрий


In [2]:
import numpy as np
import itertools


def mult(m1, m2):
    '''Умножает две матрицы xor-ом и возвращает результат
    :param:
        m1: первая матрица
        m2: вторая матрица
        
        :return: C: возвращает матрицу - xor-произведение двух матриц 
    '''
    A = np.array(m1, dtype=int)
    B = np.array(m2, dtype=int)
    C = (A@B)%2
    return np.array(C, dtype=bool)

def code_length(matr):
    '''Вычисляет кодовое расстояние массива переданных слов
    :param:
        matr: матрица - двумерный bool-массив - масиив двоичных слов
        
        :return: d: кодовое расстояние. 
    '''
    d = len(matr[0])
    for i in range(len(matr)-1):
        for j in range(i+1,len(matr)):
            # Ищем минимальное число единиц в множестве произведений i и j строк
            d = min(d,sum(np.bitwise_xor(matr[i],matr[j])))
    return d

def create_verif_matr(X):
    '''
    Создает проверочную матрицу, присоединяя к матрице X единичную матрицу снизу 
    :param X: матрица X
    :return: проверочную матрицу LinearCode
    '''
    X = X.matr
    k = X.shape[1]
    P = np.eye(k)
    return LinearCode(np.vstack([X,P]))

In [3]:
class LinearCode:
    ''' Класс линейных кодов.

    Атрибуты:\n
    matr: копия исходной матрицы\n
    k: количество строк матрицы\n
    n: количество столбцов матрицы
    
    '''
    # Конструктор
    def __init__(self, n_array):
        self.matr = np.array(n_array, dtype=bool)
        self.n = n_array.shape[1]
        self.k = n_array.shape[0]
    
    def find_first(self, line):
        '''Поиск первого ненулевого элемента в строке.
        
        :param:
        line: строка матрицы.
        
        :return: h: возвращаем позицию первого ненулевого элемента в строке. 
        '''
        # Задаем h = количеству элементов в сторке
        h = self.n
        # В строке line перебираем элементы пока не натолкнемся на 1
        for i in range(self.n):
            if self.matr[line,i]:
                h = i
                break
        # Возвращаем позицию найденной единицы в строке
        return h
    
    def xor_swap(self, line1, line2):
        '''Меняет две строки местами с помощью поэлементного XOR.

        :param:
            line1: первая строка.\n
            line2: вторая строка.

        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line1,w] ^= self.matr[line2,w]
            self.matr[line2,w] ^= self.matr[line1,w]
            self.matr[line1,w] ^= self.matr[line2,w]


    def sum(self, line1, line2):
        '''Складывает первую переданную строку со второй.

        :param:
            line1: первая строка.\n
            line2: вторая строка.
        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line2,w] ^= self.matr[line1,w]

    def __str__(self):
        '''Собирает строку (смена типа от bool к string).

        :return: s: строка с матрицей.            
        '''
        s = ''
        for i in range(self.k):
            s += '['
            for j in range(self.n):
                if self.matr[i,j]: s += "1 "
                else: s += "0 "
            s += "\b]\n"
        return s
    
    def fillrand(self):
        '''Заполняет матрицу случайным образом нулями и единицами.
        
        '''
        rand = np.random
        for i in range(self.k):
            for j in range(self.n):
                self.matr[i,j] = np.round(rand.random())

    def sort(self):
        '''Сортировка строк матрицы.
        '''
        # sort
        for i in range(self.k):
            for j in range(self.k-i-1):
                if self.find_first(j) > self.find_first(j+1):
                    self.xor_swap(j, j+1)

    def ref(self):
        '''Приводит матрицу к ступенчатому виду.
        
            ---
            Ступенчатая матрица - такая матрица, что
            * все ненулевые строки располагаются над всеми чисто нулевыми строками
            * ведущий элемент (первый, считая слева направо, ненулевой элемент строки)
            каждой ненулевой строки располагается строго правее ведущего элемента в
            строке, расположенной выше данной.
            ---
        '''
        # Проходит по всем строкм матрицы
        for i in range(self.k-1):
            # Сравнивает рабочую строку с текущей
            for j in range(i+1, self.k):
                # Запоминаем первые элементы
                first_i = self.find_first(i)
                first_j = self.find_first(j)
                # Если они равны, то делаем xor со строкой
                # Поскольку рабочая строка всегда ниже текущей, то при xor первый элемент всех нижних строк зануляется
                if first_i == first_j:
                    self.sum(i,j)
                # Если встретили строку, у которой первый элемент раньше чем у текущей, то меняем строки местами
                elif first_i > first_j:
                    self.xor_swap(i,j)
                # Иначе все хорошо, идем дальше
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
        

    def rref(self):
        '''Приводит матрицу к приведенному ступенчатому виду. 
        
            ---
            Ступенчатая матрица называется приведенной, если матрица,  не имеет нулевых строк, 
            и все ведущие элементы ее строк равны единице. При этом все элементы основных столбцов, 
            помимо ведущих элементов, являются нулями.
            
        '''
        # Сначала приводим матрицу к ступенчатому виду
        self.ref()
        # Дальше для всех строк...
        for i in range(1,self.k):
            # ...мы ищем номер ведущего элемента строки...
            ist = self.find_first(i)
            # ...и делаем xor для всех строк выше текущей
            for j in range(i-1,-1,-1):
                if(self.matr[j,ist]):
                    self.sum(i,j)
            # Таким образом зануляются все элементы над ведущим
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
    

    def delete_redundant_lines(self):
        '''Удаляет нулевые строки из матрицы.
        '''
        # Копия матрицы, с которой ведется работа
        temp = self.matr.copy()
        offset = 0 # сдвиг
        for i in range(self.k):
            # Если номер первой единицы в строке == кол-ву элементов самой строки (то есть единиц там нет)
            # Удаляем эту строку и сдвигаем указатель на 1
            if self.find_first(i) == self.n:
                temp = np.delete(temp, i - offset, 0)
                offset += 1
        # Перезаписываем матрицу на новую и очищенную
        self.matr = temp
        # Обновляем переменные-размеры матрицы
        self.n = temp.shape[1]
        self.k = temp.shape[0]

    
    def create_reduced_matr(self):
        '''
        Создает сокращенную матрицу X из текущей матрицы объекта
        :return: LinearCode(X) сокращенная матрица
        '''
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]
        return LinearCode(X)

                
    def create_verif_matr(self):
        '''Создает проверочную матрицу на основе порождающей.
        
        :return: LinearCode(H): проверочная матрица. 
        '''
        # Создаем новую матрицу в приведенном ступенчатом виде на основе текущей
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]

        # Единичная матрица
        Ind = np.identity(n = rrefed_matr.n-rrefed_matr.k, dtype = bool)

        # Создаем проверочную матрицу
        H = np.zeros(shape=(rrefed_matr.n,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        # сдвиги
        offset_i = 0
        offset_matr = 0
        # Заполняем ее значениями
        for i in range(H.shape[0]):
            # Если номер строки содержится в массиве с ведущими элементами, то заполняем строку из сокращенной матрицы
            # И прибавляем 1 к сдвигу для элементов единичной матрицы
            if i in leads:
                for j in range(H.shape[1]):
                    H[i,j] = X[i-offset_matr,j]
                offset_i += 1
            # Иначе заполняем строку из единичной матрицы
            # И прибавляем 1 к сдвику для матрицы сокращенной
            else:
                for j in range(H.shape[1]):
                    H[i,j] = Ind[i-offset_i,j]
                offset_matr += 1
        
        return LinearCode(H)
    
    def gen_codewords_summing(self):
        ''' Формирует набор слов путем сложения всех слов из порождающего множества

        :return: wordset: набор кодовых слов
        '''
        # Создаем пустой набор слов и доюавляем в него 0й элемент
        wordset = set()
        wordset.add(tuple((np.zeros(self.n, dtype=bool))))
        # Для i числа строк
        for i in range(1, self.k+1):
            # Массив длинны i всех возможных комбинаций строк 
            combinations = np.array(list(itertools.combinations(range(self.k), i)))
            for comb in combinations:
                # Для каждой комбинации
                # Создаем пустое слово
                word = np.zeros(self.n, dtype=bool)
                # Записываем в него сумму всех строк в текущей комбинации строк
                for j in comb:
                    word ^= self.matr[j]
                # Добавляем слово в набор слов
                wordset.add(tuple(word.tolist()))
        return wordset
    
    def gen_codewords_bin(self):
        ''' Формирует набор слов путем умножения всех двоичных слов длины k на G 

        :return: wordset: набор кодовых слов
        '''
        # Копия текущей матрицы
        G = LinearCode(self.matr)
        # Приводим в ступенчатому виду
        G.ref
        # Пустой набор слов
        wordset = set()
        # Набор всех возможных двоичных слов длины k
        binset = set(tuple(itertools.product((True,False),repeat=self.k)))
        # Для каждой комбинации
        for bin in binset:
            # Умножаем комбинацию на ступенчатую матрицу
            # И записываем слово в набор
            word = tuple((mult(bin,G.matr)).tolist())
            wordset.add(word)
        return wordset



### Часть 1
#### 2.1 - 2.2

In [4]:
arr2 = LinearCode(np.array([
    [1, 0, 0, 0,   1, 1, 0],
    [0, 1, 0, 0,   0 ,1 ,1],
    [0, 0, 1, 0,   1, 0, 1],
    [0, 0, 0, 1,   1, 1, 1]])) # Порождающая матрица G (7,4,3) 
print("G = ")
print(arr2)

print("X = ")
print(arr2.create_reduced_matr()) # Сокращенная матрица X

print("d =", code_length(list(arr2.gen_codewords_bin()))) # Длинна кода d

print("H = ")
H = create_verif_matr(arr2.create_reduced_matr()) # Проверочная матрица H
print(H)



G = 
[1 0 0 0 1 1 0]
[0 1 0 0 0 1 1]
[0 0 1 0 1 0 1]
[0 0 0 1 1 1 1]

X = 
[1 1 0]
[0 1 1]
[1 0 1]
[1 1 1]

d = 3
H = 
[1 1 0]
[0 1 1]
[1 0 1]
[1 1 1]
[1 0 0]
[0 1 0]
[0 0 1]



#### 2.3

In [5]:
errors = np.eye(7, dtype=bool)

# Создается словарь синдромов, под индексом берется произведение ошибки и проверочной матрицы
# Значение - соответствующая ошибка
syndromes = dict()
for i in reversed(range(7)):
    syndromes[tuple(mult(errors[i, :],H.matr))] = errors[i, :]
print("Таблица синдромов: ")
syndromes

Таблица синдромов: 


{(np.False_,
  np.False_,
  np.True_): array([False, False, False, False, False, False,  True]),
 (np.False_,
  np.True_,
  np.False_): array([False, False, False, False, False,  True, False]),
 (np.True_,
  np.False_,
  np.False_): array([False, False, False, False,  True, False, False]),
 (np.True_,
  np.True_,
  np.True_): array([False, False, False,  True, False, False, False]),
 (np.True_,
  np.False_,
  np.True_): array([False, False,  True, False, False, False, False]),
 (np.False_,
  np.True_,
  np.True_): array([False,  True, False, False, False, False, False]),
 (np.True_,
  np.True_,
  np.False_): array([ True, False, False, False, False, False, False])}

#### 2.4

In [6]:
word_K = (False, False, True, True)  # Слово длины k
word_N = mult(word_K, arr2.matr)  # Слово длины n
print("Получившееся слово: ", word_N)
print("Содержится ли это слово в списке всех слов: ", tuple(word_N) in arr2.gen_codewords_bin()) 

word_err = word_N^errors[1] # Вносим однократную ошибку
print("Слово с ошибкой: ", word_err)
print("Содержится ли это слово в списке всех слов: ", tuple(word_err) in arr2.gen_codewords_bin())

found_err = syndromes[tuple(mult(word_err, H.matr))] # Нашли ошибку по таблице синдромов
word_corr = word_err^found_err # Исправляем ошибку
print("Исправленное слово: ", word_corr)
print("Содержится ли это слово в списке всех слов: ", tuple(word_corr) in arr2.gen_codewords_bin())
print("Совпадает ли с изначальным словом: ", word_corr == word_N)

Получившееся слово:  [False False  True  True False  True False]
Содержится ли это слово в списке всех слов:  True
Слово с ошибкой:  [False  True  True  True False  True False]
Содержится ли это слово в списке всех слов:  False
Исправленное слово:  [False False  True  True False  True False]
Содержится ли это слово в списке всех слов:  True
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True]


#### 2.5

In [7]:
word_K = (False, False, True, True)  # Слово длины k
word_N = mult(word_K, arr2.matr)  # Слово длины n
print("Получившееся слово: ", word_N)
print("Содержится ли это слово в списке всех слов: ", tuple(word_N) in arr2.gen_codewords_bin()) 

word_err = word_N^errors[0]^errors[2] # Вносим двухкратную ошибку
print("Слово с ошибкой: ", word_err)
print("Содержится ли это слово в списке всех слов: ", tuple(word_err) in arr2.gen_codewords_bin())

found_err = syndromes[tuple(mult(word_err, H.matr))] # Нашли ошибку по таблице синдромов
word_corr = word_err^found_err # Исправляем ошибку
print("Исправленное слово: ", word_corr)
print("Содержится ли это слово в списке всех слов: ", tuple(word_corr) in arr2.gen_codewords_bin())
print("Совпадает ли с изначальным словом: ", word_corr == word_N)

Получившееся слово:  [False False  True  True False  True False]
Содержится ли это слово в списке всех слов:  True
Слово с ошибкой:  [ True False False  True False  True False]
Содержится ли это слово в списке всех слов:  False
Исправленное слово:  [ True  True False  True False  True False]
Содержится ли это слово в списке всех слов:  True
Совпадает ли с изначальным словом:  [False False False  True  True  True  True]


### Часть 2
#### 2.6 - 2.7

In [8]:
arr3 = LinearCode(np.array([[1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
                            [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0],
                            [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]])) # Порождающая матрица G (n,k,5) n = 11, k = 3
print("G = ")
print(arr3)

print("X = ")
print(arr3.create_reduced_matr()) # Сокращенная матрица X

print("d =", code_length(list(arr3.gen_codewords_bin()))) # Длинна кода d

print("H = ")
H = create_verif_matr(arr3.create_reduced_matr()) # Проверочная матрица H
print(H)


G = 
[1 0 0 1 0 0 0 1 1 0 1]
[0 1 0 0 1 1 1 0 1 0 0]
[0 0 1 1 1 1 0 1 1 1 1]

X = 
[1 0 0 0 1 1 0 1]
[0 1 1 1 0 1 0 0]
[1 1 1 0 1 1 1 1]

d = 5
H = 
[1 0 0 0 1 1 0 1]
[0 1 1 1 0 1 0 0]
[1 1 1 0 1 1 1 1]
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]



#### 2.8

In [9]:
# Создаем словарь синдромов
# заполняем его однократными ошибками
syndromes = dict()
errors = np.eye(11, dtype=bool)
for i in reversed(range(11)):
    syndromes[tuple(mult(errors[i, :], H.matr))] = errors[i, :]
    

# Заполняем словарь двукратными ошибками
combinations = list(itertools.combinations(range(11), 2))
# Действуем аналогично однократному заполнению, только проходим по всем двойным комбинациям однократных ошибок,
# таким образом получая все двукратные ошибки
for comb in combinations:
    errors = np.zeros(11, dtype=bool)
    for i in comb:
        errors[i] = True
    syndromes[tuple(mult(errors,H.matr))] = errors
print("Таблица синдромов: ")
syndromes

Таблица синдромов: 


{(np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.True_): array([False, False, False, False, False, False, False, False, False,
        False,  True]),
 (np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.True_,
  np.False_): array([False, False, False, False, False, False, False, False, False,
         True, False]),
 (np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.True_,
  np.False_,
  np.False_): array([False, False, False, False, False, False, False, False,  True,
        False, False]),
 (np.False_,
  np.False_,
  np.False_,
  np.False_,
  np.True_,
  np.False_,
  np.False_,
  np.False_): array([False, False, False, False, False, False, False,  True, False,
        False, False]),
 (np.False_,
  np.False_,
  np.False_,
  np.True_,
  np.False_,
  np.False_,
  np.False_,
  np.False_): array([False, False, False, False, False, False,  True, False, False,
        False, False]),
 (np.False

#### 2.9

In [10]:
errors = np.eye(11, dtype=bool)
word_K = (False, True, True)  # Слово длины k
word_N = mult(word_K, arr3.matr)  # Слово длины n
print("Получившееся слово: ", word_N)
print("Содержится ли это слово в списке всех слов: ", tuple(word_N) in arr3.gen_codewords_bin()) 

word_err = word_N^errors[4] # Внесение однократной ошибки
print("Слово с ошибкой: ", word_err)
print("Содержится ли это слово в списке всех слов: ", tuple(word_err) in arr3.gen_codewords_bin())

found_err = syndromes[tuple(mult(word_err, H.matr))] # Нашли ошибку по таблице синдромов
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Содержится ли это слово в списке всех слов: ", tuple(word_corr) in arr3.gen_codewords_bin())
print("Совпадает ли с изначальным словом: ", word_corr == word_N)

Получившееся слово:  [False  True  True  True False False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  True
Слово с ошибкой:  [False  True  True  True  True False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  False
Исправленное слово:  [False  True  True  True False False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  True
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True  True  True  True  True]


#### 2.10

In [11]:
errors = np.eye(11, dtype=bool)
word_K = (False, True, True)  # Слово длины k
word_N = mult(word_K, arr3.matr)  # Слово длины n
print("Получившееся слово: ", word_N)
print("Содержится ли это слово в списке всех слов: ", tuple(word_N) in arr3.gen_codewords_bin()) 

word_err = word_N^errors[4]^errors[5] # Внесение двукратной ошибки
print("Слово с ошибкой: ", word_err)
print("Содержится ли это слово в списке всех слов: ", tuple(word_err) in arr3.gen_codewords_bin())

found_err = syndromes[tuple(mult(word_err, H.matr))] # Нашли ошибку по таблице синдромов
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Содержится ли это слово в списке всех слов: ", tuple(word_corr) in arr3.gen_codewords_bin())
print("Совпадает ли с изначальным словом: ", word_corr == word_N)

Получившееся слово:  [False  True  True  True False False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  True
Слово с ошибкой:  [False  True  True  True  True  True  True  True False  True  True]
Содержится ли это слово в списке всех слов:  False
Исправленное слово:  [False  True  True  True False False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  True
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True  True  True  True  True]


#### 2.11

In [12]:
errors = np.eye(11, dtype=bool)
word_K = (False, True, True)  # Слово длины k
word_N = mult(word_K, arr3.matr)  # Слово длины n
print("Получившееся слово: ", word_N)
print("Содержится ли это слово в списке всех слов: ", tuple(word_N) in arr3.gen_codewords_bin()) 

word_err = word_N^errors[4]^errors[5]^errors[6] # Внесение трехкратной ошибки
print("Слово с ошибкой: ", word_err)
print("Содержится ли это слово в списке всех слов: ", tuple(word_err) in arr3.gen_codewords_bin())

found_err = syndromes[tuple(mult(word_err, H.matr))] # Нашли ошибку по таблице синдромов
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Содержится ли это слово в списке всех слов: ", tuple(word_corr) in arr3.gen_codewords_bin())
print("Совпадает ли с изначальным словом: ", word_corr == word_N)

Получившееся слово:  [False  True  True  True False False  True  True False  True  True]
Содержится ли это слово в списке всех слов:  True
Слово с ошибкой:  [False  True  True  True  True  True False  True False  True  True]
Содержится ли это слово в списке всех слов:  False
Исправленное слово:  [False False  True  True  True  True False  True  True  True  True]
Содержится ли это слово в списке всех слов:  True
Совпадает ли с изначальным словом:  [ True False  True  True False False False  True False  True  True]
